Author: Megan Tabbutt

Latest version: 03_25_21

Notes:

Resources: 

- PyPI: https://pypi.org/project/CensusData/
- Documentation: https://jtleider.github.io/censusdata/
- County level data: https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html


Datasets:

    ACS 5-year estimates (2005-2009 to 2015-2019),
    ACS 1-year estimates (2012-2019),
    ACS 3-year estimates (2010-2012 to 2011-2013),
    ACS 1-year supplemental estimates (2014-2019),
    Census 2010 Summary File 1.


## Questions:

_1. What assumptions can we make to not do an $n^2+$ type loop on finding counties?_
- Can we assume it is confined to one state? 
- Will finding NSEW points to check for counties be sufficent? Add a middle point too? 

_2. What resolution do we need for the county level data?_


In [1]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import censusdata
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Feature #1: connect gtfs files to pull counties from CensusData:

In [3]:
# Pull in the latitude and longitude from gtfs data 
gtfs_data_path = "data/mmt_gtfs/"
lat_lon_file = 'stops.csv'
lat_lon_df = pd.read_csv(gtfs_data_path + lat_lon_file)
lat_lon_df.head(3)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,agency_id,jurisdiction_id,location_type,parent_station,relative_position,cardinal_direction,wheelchair_boarding,primary_street,address_range,cross_location
0,1662,1662,E Gorham & N Blount (WB),This stop (#1662) is westbound on the 698 bloc...,43.082455,-89.380990,MMT,CMAD,0,NaN,1,270.0,1,E Gorham,698,N Blount
1,2437,2437,Toepfer & Birch (NB),This stop (#2437) is northbound on the 441 blo...,43.058547,-89.441740,MMT,CMAD,0,NaN,1,0.0,1,Toepfer,441,Birch
2,7931,7931,North Star & Orion (NB),This stop (#7931) is northbound on the 741 blo...,43.087951,-89.279047,MMT,CMAD,0,NaN,3,0.0,1,North Star,741,Orion


### make into a geoDF if you want to plot below to check 
gdf = geopandas.GeoDataFrame(lat_lon_df, geometry=geopandas.points_from_xy(lat_lon_df.shape_pt_lon, lat_lon_df.shape_pt_lat))
gdf.head(3)

In [4]:
# Convert to shapely point objects in tuples
ID_points = [Point(lat_lon_df['stop_lon'][i], lat_lon_df['stop_lat'][i]) for i in range(len(lat_lon_df))]
ID_points[0].coords[:]

[(-89.38099, 43.082454999999996)]

### Trying to reduce the $n^2$ problem

Sol 1: find the NESW farthest points, if all inside the same county you are good. WRONG - logic error. 

Sol 2: What I am doing below, write description. 

In [5]:
# traverse the whole list? or will it be confined to one state? 
countyData = geopandas.read_file("/Users/megantabbutt/Downloads/cb_2018_us_county_20m/cb_2018_us_county_20m.shp")
countyData.head(3)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,37,017,01026336,0500000US37017,37017,Bladen,06,2265887723,33010866,"POLYGON ((-78.90200 34.83527, -78.79960 34.850..."
1,37,167,01025844,0500000US37167,37167,Stanly,06,1023370459,25242751,"POLYGON ((-80.49737 35.20210, -80.29542 35.502..."
2,39,153,01074088,0500000US39153,39153,Summit,06,1069181981,18958267,"POLYGON ((-81.68699 41.13596, -81.68495 41.277..."


In [6]:
uniquePoints = ID_points.copy()
counties = []

startingPoint = uniquePoints[0]
startingPoint

for county in countyData.iterrows():
    if county[1]['geometry'].contains(startingPoint):
        startingCounty = county[1]['NAME']
        startingState = county[1]['STATEFP']
        print(startingCounty, " = Starting County")
        print(startingState, " = Starting State")
        
stateDF = countyData[countyData['STATEFP'] == startingState]
countyDF = stateDF[stateDF['NAME'] == startingCounty]

print(len(uniquePoints))

# Need to make this piece of code work!
remainingPoints = []
if len(counties) >= 0:
    for point in uniquePoints:
        for county in countyDF.iterrows():
            if county[1]['geometry'].contains(point):
                if county[1]['NAME'] not in counties:
                    counties.append(county[1]['NAME'])
            else:
                remainingPoints.append(point)
            break
   
# FIX THIS CODE
if remainingPoints != 0:
    print()
    
    
len(remainingPoints)

Dane  = Starting County
55  = Starting State
2135



0